# Python Boot Camp

***

## Table of contents:

* [Images as arrays](#first)   
* [Image color space](#second)
* [Image data types](#third)
* [Plotting images](#fourth)
* [Images in Deep Learning frameworks](#fifth)
* [Simple image manipulation](#sixth)
* [Loading a set of images](#seventh)
* [Segmentation maps](#eighth)
* [Batching](#ninth)
* [Convolutions](#tenth)
* [Data augmentation](#eleventh)


*** 

Welcome!
In this notebook, we will go through some basic image processing in Python and familiarize ourselves with different utilities that can be useful for any Deep Learning pipeline, utilities provides through libraries like `skimage`, `imgaug`, `glob`, `tqdm` and more.

We will be using sample images from this **[three-dimensional X-ray microtomography thalamocortical dataset](https://github.com/nerdslab/xray-thc)**, used to characterize brain heterogeneity. These samples, imaged in the Striatum and Hypothalamus regions of a mouse brain, were annotated to get microstructure segmentation maps (of cell bodies, blood vessels, and myelinated axons). The full dataset is available on [bossdb](https://bossdb.org/project/prasad2020)! 

<div class="alert alert-danger">
    <b>Set your python kernel to <code>00-boot</code></b>
</div>

In [ ]:
import matplotlib.pyplot as plt 
import numpy as np

## Images as arrays <a class="anchor" id="first"></a>

Images are represented as numpy arrays of shape (height, width, channels).

![RGB image as a numpy array](./assets/image_as_array.png)

<div style="text-align: right"> Credit: <a href="https://e2eml.school/convert_rgb_to_grayscale.html">Brandon Rohrer’s Blog</a></div>


Multiple utilities/packages exist to read images from files in Python,
we will use `skimage.io.imread`.


If you look in the directory containing this notebook, you will find a folder called data which includes some tiff files. 
Here our images have the .tiff extension and all start with "img".

Let's load one image.

In [ ]:
from skimage import io

img = io.imread('data/img_650_1162__600_1112__0.tiff')
print('Type:', type(img))

## Image color space <a class="anchor" id="second"></a>
If the image is in grayscale, then the number of channels is equal to 1,
in which case the array can also be of shape (height, width).
If the image is RGB, then the number of channels is 3
with each channel encoding the red, green and blue components.

<div class="alert alert-block alert-warning"> 
<b>Is <code>img</code> RGB or grayscale ?</b></div>

<div style="text-align: right"> Help: <a href="https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf">Numpy cheatsheet</a></div>

<div class="alert alert-block alert-info"> <b>Reshape <code>img</code> such that its shape is <code>(height, width, 1)</code> </b></div>

<div style="text-align: right"> Hint: <a href="https://numpy.org/doc/stable/reference/generated/numpy.expand_dims.html">expand dims</a></div>

In [ ]:
img = ...

## Image data types <a class="anchor" id="third"></a>


Images can be represented by a variety of data types. The following is a list of the most common datatypes:
- `bool`: binary, 0 or 1
- `uint8`: unsigned integers, 0 to 255 range
- `float`: -1 to 1 or 0 to 1

<div class="alert alert-block alert-success">
<b> What is the data type of <code> img</code>? What are the minimum and maximum intensity values? </b>
</div>

<div style="text-align: right"> Help: <a href="https://s3.amazonaws.com/assets.datacamp.com/blog_assets/Numpy_Python_Cheat_Sheet.pdf">Numpy cheatsheet</a></div>

## Plotting images <a class="anchor" id="fourth"></a>

Using `matplotlib`, we can visualize the image. <br> When the image is in grayscale,
a colormap can be specified using the `cmap` argument. <br> By default,
the colormap is `viridis`, in the following cell we will use `gray`.

> Useful `matplotlib` resources:
> - [matplotlib cheatsheets](https://github.com/matplotlib/cheatsheets)
> - [colormaps in matplotlib](https://matplotlib.org/stable/tutorials/colors/colormaps.html)

In [ ]:
plt.imshow(img, cmap='gray')
plt.axis('off')
plt.show()

## Images in Deep Learning frameworks <a class="anchor" id="fifth"></a>


In pytorch, tensorflow or jax (ML libraries that we will soon be using), images are represented as (channels, height, width)
and are rescaled to be in the [0, 1] range.

<div class="alert alert-block alert-info"> 
    <b>Generate a new image that respects these conventions.</b> You can use <code>np.transpose</code></div>

<div style="text-align: right"> Help: <a href="https://numpy.org/doc/stable/reference/generated/numpy.transpose.html">Numpy Transpose</a></div>

In [ ]:
img_tensor = ...

## Simple image manipulation <a class="anchor" id="sixth"></a>

Given that images are numpy arrays, we can take advantage of the powerful
 [indexing](https://numpy.org/doc/stable/reference/arrays.indexing.html)
 to perform simple transformations like cropping, downsampling and flipping.

In [ ]:
# cropping
cropped_img = img[10:310, 50:350]

# downsampling
factor = 2
downsampled_img = img[::factor, ::factor]

# horizontal flip
hflip_img = img[:,::-1]

fig, axs = plt.subplots(1, 4, figsize=(15,5), frameon=False)

axs[0].imshow(img, cmap='gray')
axs[0].set_title('Original image\nSize = {}'.format(str(img.shape)))
axs[0].axis('off')

axs[1].imshow(cropped_img, cmap='gray')
axs[1].set_title('Cropped image\nSize = {}'.format(str(cropped_img.shape)))
axs[1].axis('off')

axs[2].imshow(downsampled_img, cmap='gray')
axs[2].set_title('Downsampled image\nSize = {}'.format(str(downsampled_img.shape)))
axs[2].axis('off')

axs[3].imshow(hflip_img, cmap='gray')
axs[3].set_title('Horizontal Flip\n Size = {}'.format(str(hflip_img.shape)))
axs[3].axis('off')
plt.show()

<div class="alert alert-block alert-info"> 
<b> Apply the following transformations:</b> <br>
<li>Center crop of size <code>(256, 256)</code></li>
<li>Vertical Flip</li>
</div>

<div class="alert alert-block alert-success"><h1>Checkpoint 1</h1>
 
</div>

## Loading a set of images <a class="anchor" id="seventh"></a>

Given a set of images in a folder, we need to be able to easily find the pathnames and load them in. <br>
`glob` is a standard package that provides a utility for finding all pathnames that match a given pattern.

Here our images have the `.tiff` extension and all names start with `img`

In [ ]:
import os
from glob import glob

root = 'data/'
img_filenames = glob(os.path.join(root, 'img*.tiff'))

print('Found:')
for img_filename in img_filenames:
    print(' ', img_filename)

We can now load the images. <br>
We will use `tqdm` to track progress (even though we only have a small number of images here). <br> `tqdm` provides a progress bar that simply wraps around any iterable, making it useful for tracking training progress, for example.

In [ ]:
from tqdm import tqdm

imgs = []
for fname in tqdm(img_filenames):
    img = io.imread(fname)
    img = np.expand_dims(img, axis=2)
    imgs.append(img)

fig, axs = plt.subplots(1, 4, figsize=(15,5))
for i in range(4):
    axs[i].imshow(imgs[i], cmap='gray')
    axs[i].axis('off')

## Segmentation maps <a class="anchor" id="eighth"></a>

We often want to "segment" data - i.e: assign labels to parts of images. <br>
This dataset has already been segmented for us into four categories: cell bodies, blood vessels, axons, as well as background. <br>
For each image file, you will find the corresponding segmentation file in the data folder. These begin with "annos" and have the .tiff extension. <br>

<div class="alert alert-block alert-info"> 
<b>Using what we have seen thus far, load the segmentation maps into a list called <code>seg_maps</code> and visualize them. <br>Be careful with the file ordering.
</b>
</div>



<div style="text-align: right"> <a href="https://pythonbasics.org/replace/">Hint</a></div>

In [ ]:
seg_filenames = ...

seg_maps = ...

You will notice that there are 4 unique values in the segmentation maps, each corresponding to one of these four classes:
- 0: background
- 1: cell
- 2: blood vessel
- 3: axon


<div class="alert alert-block alert-warning"> 
<b>What is the data type of these segmentation maps?</b>
<b>What data types do you know of? How should one choose what data type to use? <br> Should segmentation maps necessarily be saved in the same data type as their respective images? If yes, why? If no, why not?</b>
</div>


### Visualizing Segmentation Maps

Let's visualize the segmentation map of one image.

In [ ]:
seg_map_example = seg_maps[0]

labels = ['background', 'cells', 'blood vessels', 'axons']

fig, axs = plt.subplots(1, 4, figsize=(15,5))
for i in range(4):
    tmp = np.zeros_like(seg_map_example)
    tmp[seg_map==i] = 1
    axs[i].imshow(tmp, cmap='gray')
    axs[i].set_title(labels[i])
    axs[i].axis('off')

## Batching <a class="anchor" id="ninth"></a>

In ML/DL, we often have to deal with very large datasets. It soon becomes inefficient to process all the data at once, so it's useful to split the data into "batches" that we can process individually. So for purely reasons of computational cost, this is often useful.

We will also see another reason for which batching can be useful - for instance when running gradient descent on non-convex landscapes. Here computing the gradient on a subset of the data gives us an approximate/noisy gradient making it less likely for us to end up being stuck in local minima. This is what we call "stochastic gradient descent".

Let us make our first batch of images, containing $B$ number of images. 
The shape of the batch will thus get an additional "batch dimension" at the first dimension, i.e. (batch_size, channels, height, width).

**Q: Make a batch out of the four images (this will be a 4D numpy array)**


<div style="text-align: right"> <a href="https://numpy.org/doc/stable/reference/generated/numpy.stack.html">Hint</a></div>

In [ ]:
batch = ...

<div class="alert alert-block alert-success"><h1>Checkpoint 2</h1>
 
</div>

## Convolutions <a class="anchor" id="tenth"></a>

Convolutions are the elementary operations used in CNNs. The image (and later, the feature maps) are convolved with multiple kernels whose weights are learned. Below is a visual of the pixel values in the output matrix (green) being computed from neighboring pixels in the input matrix (blue). 

![](https://raw.githubusercontent.com/vdumoulin/conv_arithmetic/master/gif/no_padding_no_strides.gif)

<div style="text-align: right"> Credit: <a href="https://github.com/vdumoulin/conv_arithmetic">Vincent Dumoulin, Francesco Visin</a></div>

<div class="alert alert-block alert-info"> 
<b>
    Implement a function that performs "convolution". Assume that your image is square and that your kernel is square and has an odd width. <br>Note that your output image will be smaller (we won't use padding for now). </b>
</div>

In [ ]:
def conv2d(img, kernel):
    m = img.shape[0]  #size of original image
    k = kernel.shape[0]  #size of filter
    
    mc = ... #size of the new image after convolution
    # NB: there's no one right answer
    
    conv_img = np.zeros((mc,mc))
    
    # perform convolution
    for ii in range(mc):
        for jj in range(mc):
            conv_img[ii,jj] = ...
            # see Florian's slide for hint
            
    
    return conv_img

### Smoothing filter
Convolving the image with a smoothing filter is equivalent to replacing the value of each pixel with the average pixel value within a window of size $d \times d$ around it.

<div class="alert alert-block alert-info"> 
<b> Design a smoothing filter. Try different values of <code>d</code>.</b>
</div>

In [ ]:
kernel = ... # try a kernel size of 10, then try some different values

In [ ]:
img = imgs[0][:,:,0] # choose one example image
smoothed_img = ...

### Compare the original and smoothed image

In [ ]:
fig, axs = plt.subplots(ncols = 2)
axs[0].imshow(img, cmap='gray')
axs[0].set_title('original')
axs[1].imshow(smoothed_img, cmap='gray')
axs[1].set_title('smoothed image')

### Sobel filter
The following is known as the Sobel filter:

$$
\begin{bmatrix}
    1 & 2 & 1 \\
    0 & 0 & 0 \\
    -1 & -2 & -1 
\end{bmatrix}
$$
<div class="alert alert-block alert-warning"> 
<b>What pattern would produce the largest output when convolved with this filter? (assuming the sum of the pattern is capped)</b>
</div>
<div class="alert alert-block alert-info"> 
<b>Apply the Sobel filter and describe what it does</b>
</div>

In [ ]:
kernel = ... 

<div class="alert alert-block alert-success"><h1>Checkpoint 3</h1>
 
</div>

## Data augmentation <a class="anchor" id="eleventh"></a>

Having collected your hard earned data you want to make the most of it. In ML/DL, we're often limited by the size of our the training set. How could we artificially inflate our data to provide more input to our model and help it generalize better?

One trick is to make simple transformations to our data such as rotating it or adding noise - this process is generally called "data augmentation" (DA) and is widely used in ML.  

`imgaug` is a Python library that provides a very extensive set of image augmentations,
and that seamlessly handles complex annotations like segmentation maps, bounding boxes or keypoints. Let us import `imgaug` in the cell below.

In [ ]:
import imgaug as ia
from imgaug import augmenters as iaa

# fix the random seed
ia.seed(4)

#### Augmenting an image
To use an augmentation, we can instantiate an `Augmenter` with a set of hyperparameters.
With affine transforamtions for example, we can specify the range of the rotation angle to be `(-45, 45)` degrees.

In `imgaug`, the channel-axis is always expected to be the last axis and may be skipped for grayscale images. It is
also recommended to work with the `uint8` dtype.

In [ ]:
rotate = iaa.Affine(rotate=(-45, 45))
img_aug = rotate(image=imgs[0])

plt.imshow(img_aug, cmap='gray')
plt.axis('off')
plt.show()

Note: Try running the previous cell again.

#### Augmenting image AND segmentation map

When applying certain augmentations, we want to make sure that the segmentation map is changed accordingly.

In [ ]:
from imgaug.augmentables.segmaps import SegmentationMapsOnImage

# convert array to SegmentationMapsOnImage instance
seg_map = SegmentationMapsOnImage(seg_maps[0], shape=imgs[0].shape)
# augment
img_aug, seg_map_aug = rotate(image=imgs[0], segmentation_maps=seg_map)

fig, axs = plt.subplots(1, 2, figsize=(8,5))
axs[0].imshow(img_aug, cmap='gray')
axs[0].axis('off')

axs[1].imshow(seg_map_aug.draw()[0], cmap='gray')
axs[1].axis('off')
plt.show()

#### Applying multiple augmentations
We can compose multiple augmentations

In [ ]:
seq = iaa.Sequential([
    iaa.AdditiveGaussianNoise(scale=(0, 30)),
    iaa.pillike.FilterEdgeEnhanceMore(),
    iaa.Crop(percent=(0., 0.2))
])

imgs_aug = seq(images=imgs)

fig, axs = plt.subplots(2, 1, figsize=(20,10))
axs[0].imshow(np.concatenate((imgs), axis=1), cmap='gray')
axs[0].axis('off')

axs[1].imshow(np.concatenate((imgs_aug), axis=1), cmap='gray')
axs[1].axis('off')
plt.show()


<div class="alert alert-block alert-info"> 
<b>Familiarize yourself with the different augmentations available through <code>imgaug</code>. <br>
Refer to the <a href = "https://github.com/aleju/imgaug">examples</a> and the <a href="https://imgaug.readthedocs.io/en/latest/">documentation</a>. Identify and apply augmentations that you think are interesting.
</b>
</div>